In [1]:
import findspark
findspark.init("/opt/manual/spark/")

In [2]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [3]:
#start-all.sh
spark = SparkSession.builder \
.appName("Case Study 1") \
.master("yarn") \
.enableHiveSupport() \
.getOrCreate()

In [4]:
df = spark.read \
.option("header", True) \
.option("inferSchema", True) \
.csv("file:///home/train/datasets/dirty_store_transactions.csv")

In [5]:
df.show(5)

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
only showing top 5 rows



In [6]:
df.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: string (nullable = true)



In [7]:
df.count()

37853

In [8]:
len(df.columns)

9

In [17]:
df1 = df.withColumn("Date", F.to_date(F.col("Date"), "yyyy-MM-dd")) \
.withColumn("STORE_LOCATION", F.regexp_replace(F.col("STORE_LOCATION"), "[^A-Za-z0-9]","")) \
.withColumn("PRODUCT_ID", F.regexp_replace(F.col("PRODUCT_ID"), "[^A-Za-z0-9]",""))

In [18]:
df1.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: string (nullable = true)
 |-- CP: string (nullable = true)
 |-- DISCOUNT: string (nullable = true)
 |-- SP: string (nullable = true)
 |-- Date: date (nullable = true)



In [23]:
df2 = df1.withColumn("MRP", F.regexp_replace(F.col("MRP"),"\$","").cast(FloatType())) \
.withColumn("CP", F.regexp_replace(F.col("CP"),"\$","").cast(FloatType())) \
.withColumn("DISCOUNT", F.regexp_replace(F.col("DISCOUNT"),"\$","").cast(FloatType())) \
.withColumn("SP", F.regexp_replace(F.col("SP"),"\$","").cast(FloatType()))

In [24]:
df2.printSchema()

root
 |-- STORE_ID: string (nullable = true)
 |-- STORE_LOCATION: string (nullable = true)
 |-- PRODUCT_CATEGORY: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- MRP: float (nullable = true)
 |-- CP: float (nullable = true)
 |-- DISCOUNT: float (nullable = true)
 |-- SP: float (nullable = true)
 |-- Date: date (nullable = true)



In [25]:
df2.limit(3).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,NewYork,Electronics,12254943,31.0,20.77,1.86,29.139999,2019-11-26
1,YR7220,NewYork,Furniture,72619323C,15.0,9.75,1.50,13.500000,2019-11-26
2,YR7220,NewYork,Electronics,34161682B,88.0,62.48,4.40,83.599998,2019-11-26


In [39]:
spark.sql("show databases;").show()

+---------+
|namespace|
+---------+
|bookstore|
|  default|
|movielens|
|    test1|
+---------+



In [42]:
spark.sql("show tables;").show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|order_items_tbl|      false|
| default|     orders_tbl|      false|
+--------+---------------+-----------+



In [44]:
spark.sql("use test1;")
spark.sql("show tables;").show()

+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
|   test1|       advertising|      false|
|   test1|   adv_sales_gt_20|      false|
|   test1|             churn|      false|
|   test1|clean_transactions|      false|
|   test1|               flo|      false|
|   test1|           flo_orc|      false|
|   test1|     hotels_bucket|      false|
|   test1|        hotels_orc|      false|
|   test1|    hotels_parquet|      false|
|   test1|        hotels_prt|      false|
|   test1|hotels_text_snappy|      false|
|   test1|        hotels_txt|      false|
|   test1|    mytable_backup|      false|
|   test1|   mytable_renamed|      false|
+--------+------------------+-----------+



In [26]:
df2.write \
.format("orc") \
.mode("overwrite") \
.saveAsTable("test1.clean_transactions")

In [27]:
! hdfs dfs -ls -h /user/hive/warehouse/test1.db/clean_transactions

Found 2 items
-rw-r--r--   1 train hive          0 2023-08-22 15:31 /user/hive/warehouse/test1.db/clean_transactions/_SUCCESS
-rw-r--r--   1 train hive    187.6 K 2023-08-22 15:31 /user/hive/warehouse/test1.db/clean_transactions/part-00000-62ea048f-2d55-4201-927b-004698cc7859-c000.snappy.orc


In [ ]:
spark.sql("show tables;").show()

In [45]:
jdbcURL = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [47]:
df2.write \
.jdbc(url=jdbcURL, table= "clean_transactions", mode="overwrite" ,properties={"driver":"org.postgresql.Driver"})

In [51]:
spark.sql("select count(*) from test1.clean_transactions").show()

+--------+
|count(1)|
+--------+
|   37853|
+--------+



In [53]:
df2.write \
.format("parquet") \
.mode("overwrite") \
.option("compression","snappy") \
.save("hdfs://localhost:9000/user/train/spark_case_study_1_transactions")